# 第3章 反復最適化

LLMを使用してアプリケーションを構築する際、実践的な面では*最初の試行*で最終アプリケーションに適した Prompt を成功裏に取得することは困難です。しかし、これは重要ではありません。プロンプトを継続的に改善する良い反復プロセスがあれば、タスクに適したプロンプトを得ることができます。機械学習モデルの訓練と比較して、プロンプトの面では一度で成功する確率がやや高いかもしれませんが、上述のとおり、プロンプトが一度で完璧になるかどうかは重要ではありません。最も重要なのは、**段階的な反復**によってアプリケーションに効果的なプロンプトを見つけるプロセスです。

そこで本章では、製品仕様書からマーケティングコピーを生成することを例として、いくつかのプロセスフレームワークを示し、プロンプトを段階的に反復して分析・改善する方法について考えるよう促します。

アンドリュー・ン（Andrew Ng、原チュートリアルの著者）の機械学習コースで示された図表では、機械学習開発のフローが説明されています。通常、まずアイデアがあり、次に以下のフローで実装します：コードを書き、データを取得し、モデルを訓練し、実験結果を得る。その後、結果を確認し、誤差やエラーを分析し、適用領域を見つけ、具体的な問題に対する具体的な思考や解決方法を変更することもできます。その後、再び実装を変更し、別の実験を実行するなど、反復を繰り返し、最終的に効果的な機械学習モデルを得ます。LLMベースのアプリケーションのプロンプトを書く際、プロセスは非常に似ている可能性があります。完了したいタスクについてのアイデアが生まれた後、最初のプロンプトを書くことを試み、前章で述べた2つの原則を満たすよう注意してください：**明確で具体的であり、システムに十分な思考時間を与える**。その後、実行して結果を確認できます。最初の効果が良くない場合、反復プロセスは指示が十分に明確でない理由や、アルゴリズムに十分な思考時間を与えていない理由を見つけ、アイデアやプロンプトを改善するなどを行い、アプリケーションに適したプロンプトを見つけるまで何度もサイクルを繰り返すことです。

世界のすべてに適用できるいわゆる「最適なプロンプト」を見つけることは困難で、より良い方法は効果的な反復プロセスを見つけることです。これにより、アプリケーションに適したプロンプトを迅速に見つけることができます。

<div class="toc">
    <ul class="toc-item">
        <li><span><a href="#一環境設定" data-toc-modified-id="一、環境設定">一、環境設定</a></span></li>
        <li>
            <span><a href="#二タスク製品仕様書からマーケティング製品説明を生成" data-toc-modified-id="二、タスク——製品仕様書からマーケティング製品説明を生成">二、タスク——製品仕様書からマーケティング製品説明を生成</a></span>
            <ul class="toc-item">
                <li><span><a href="#21-問題一生成テキストが長すぎる" data-toc-modified-id="2.1 問題一：生成テキストが長すぎる">2.1 問題一：生成テキストが長すぎる</a></span></li>
                <li><span><a href="#22-問題二間違ったテキスト詳細を捉える" data-toc-modified-id="2.2 問題二：間違ったテキスト詳細を捉える">2.2 問題二：間違ったテキスト詳細を捉える</a></span></li>
                <li><span><a href="#23-問題三表形式の説明を追加" data-toc-modified-id="2.3 問題三：表形式の説明を追加">2.3 問題三：表形式の説明を追加</a></span></li>
            </ul>
        </li>
    </ul>
</div>

## 一、環境設定

前章と同様に、まずOpenAI APIを使用する環境を設定する必要があります

In [ ]:
import openai
# サードパーティライブラリのインポート

openai.api_key = "sk-..."
# API_KEYの設定、ご自身のAPI_KEYに置き換えてください

In [ ]:
# OpenAI APIをラップする関数、パラメータはプロンプト、対応する結果を返す
def get_completion(prompt, model="gpt-3.5-turbo"):
    '''
    prompt: 対応するプロンプトテキスト
    model: 呼び出すモデル、デフォルトはgpt-3.5-turbo(ChatGPT)、ベータ版資格を持つユーザーはgpt-4を選択可能
    '''
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # モデル出力の温度係数、出力のランダム性を制御
    )
    # OpenAIのChatCompletion APIを呼び出し
    return response.choices[0].message["content"]

## 二、タスク——製品仕様書からマーケティング製品説明を生成

椅子の仕様書が与えられます。説明では*中世紀インスピレーション*シリーズに属し、イタリア製であり、材料、構造、寸法、オプション付属品などのパラメータが紹介されています。この仕様書を使用してマーケティングチームがECプラットフォーム向けのマーケティング説明文を作成することを支援したいと仮定します：

In [3]:
# 示例：产品说明书
fact_sheet_chair = """
OVERVIEW
- Part of a beautiful family of mid-century inspired office furniture, 
including filing cabinets, desks, bookcases, meeting tables, and more.
- Several options of shell color and base finishes.
- Available with plastic back and front upholstery (SWC-100) 
or full upholstery (SWC-110) in 10 fabric and 6 leather options.
- Base finish options are: stainless steel, matte black, 
gloss white, or chrome.
- Chair is available with or without armrests.
- Suitable for home or business settings.
- Qualified for contract use.

CONSTRUCTION
- 5-wheel plastic coated aluminum base.
- Pneumatic chair adjust for easy raise/lower action.

DIMENSIONS
- WIDTH 53 CM | 20.87”
- DEPTH 51 CM | 20.08”
- HEIGHT 80 CM | 31.50”
- SEAT HEIGHT 44 CM | 17.32”
- SEAT DEPTH 41 CM | 16.14”

OPTIONS
- Soft or hard-floor caster options.
- Two choices of seat foam densities: 
medium (1.8 lb/ft3) or high (2.8 lb/ft3)
- Armless or 8 position PU armrests 

MATERIALS
SHELL BASE GLIDER
- Cast Aluminum with modified nylon PA6/PA66 coating.
- Shell thickness: 10 mm.
SEAT
- HD36 foam

COUNTRY OF ORIGIN
- Italy
"""

In [4]:
#   Prompt ：基于说明书生成营销描述
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)


Introducing our stunning mid-century inspired office chair, the perfect addition to any home or business setting. Part of a beautiful family of office furniture, including filing cabinets, desks, bookcases, meeting tables, and more, this chair is available in several options of shell color and base finishes to suit your style. Choose from plastic back and front upholstery (SWC-100) or full upholstery (SWC-110) in 10 fabric and 6 leather options.

The chair is constructed with a 5-wheel plastic coated aluminum base and features a pneumatic chair adjust for easy raise/lower action. It is available with or without armrests and is qualified for contract use. The base finish options are stainless steel, matte black, gloss white, or chrome.

Measuring at a width of 53 cm, depth of 51 cm, and height of 80 cm, with a seat height of 44 cm and seat depth of 41 cm, this chair is designed for ultimate comfort. You can also choose between soft or hard-floor caster options and two choices of seat fo

In [ ]:
# 例：製品仕様書
fact_sheet_chair = """
概要

    美しい中世紀風オフィス家具シリーズの一部で、ファイルキャビネット、デスク、本棚、会議テーブルなどが含まれます。
    複数のシェルカラーとベース仕上げオプション。
    プラスチック製の前後背もたれ装飾（SWC-100）または10種類のファブリックと6種類のレザーでのフル装飾（SWC-110）が利用可能。
    ベース仕上げオプション：ステンレス鋼、マットブラック、グロスホワイト、またはクロム。
    椅子はアームレスト付きまたはなしで利用可能。
    家庭用またはビジネス環境に適しています。
    契約使用の資格があります。

構造

    5輪プラスチックコーティングアルミベース。
    昇降操作が簡単な空気圧椅子調整。

寸法

    幅53cm | 20.87インチ
    奥行き51cm | 20.08インチ
    高さ80cm | 31.50インチ
    シート高44cm | 17.32インチ
    シート奥行き41cm | 16.14インチ

オプション

    ソフトフロアまたはハードフロアキャスターオプション。
    2種類のシートフォーム密度：中程度（1.8ポンド/立方フィート）または高密度（2.8ポンド/立方フィート）。
    アームレストなしまたは8ポジションPUアームレスト。

材料
シェルベースグライダー

    改質ナイロンPA6/PA66コーティングを施した鋳造アルミニウム。
    シェル厚：10mm。
    シート
    HD36フォーム

原産国

    イタリア
"""

In [ ]:
#   Prompt ：仕様書に基づいてマーケティング説明を作成
prompt = f"""
あなたのタスクは、技術仕様書に基づいて製品の小売ウェブサイト用説明を作成することで、マーケティングチームを支援することです。

三つのバッククォートで区切られた技術仕様書で提供された情報に基づいて、製品説明を書いてください。

技術仕様書: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

## 2.1 問題一：生成テキストが長すぎる

技術仕様書から製品説明を書くという要求を非常によく完了しているようで、精緻な中世紀風オフィスチェアが紹介されています。しかし、これを見たとき、長すぎると感じます。

そこで上記のプロセスで、アイデアを生成した後にプロンプトを書き、結果を得ましたが、長すぎるため満足していません。そこでプロンプトを明確化し、生成テキストの長さを制限し、最大50語まで使用するよう求めます。

In [21]:
# 优化后的 Prompt，要求生成描述不多于 50 词
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)


Introducing our beautiful medieval-style office furniture collection, including filing cabinets, desks, bookcases, and conference tables. Choose from a variety of shell colors and base coatings, with optional plastic or fabric/leather decoration. The chair features a plastic-coated aluminum base with five wheels and pneumatic height adjustment. Perfect for home or commercial use. Made in Italy.


提取回答并根据空格拆分，答案为54个字，较好地完成了设计要求。

In [22]:
lst = response.split()
print(len(lst))

54


In [ ]:
# 最適化後のプロンプト、生成する説明を50語以内に要求
prompt = f"""
あなたのタスクは、技術仕様書に基づいて製品の小売ウェブサイト用説明を作成することで、マーケティングチームを支援することです。

三つのバッククォートで区切られた技術仕様書で提供された情報に基づいて、製品説明を書いてください。

最大50語まで使用してください。

技術仕様：```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

In [ ]:
# 日本語は単語分割が必要なため、ここでは全体の長さを直接計算
len(response)

LLMは厳格な文字数制限をかろうじて処理できますが、正確には実現されません。この例では、英語出力で50語以内の制御を要求していますが、時には60または65語の内容を出力することがありますが、これもまだ合理的です。理由は、LLMがテキストを解釈するためにトークナイザー（tokenizer）を使用しているが、文字数の計算においてはそれほど優れた性能を示さないことです。得られる出力の長さを制御しようとする多くの異なる方法があります（いくつかの文/語/漢字/文字 (characters) など）。

## 2.2 問題二：間違ったテキスト詳細を捉える

この宣伝文を継続的に改善していくと、第二の問題が見つかります。このウェブサイトは消費者に直接販売しているのではなく、実際には家具小売業者を対象としており、彼らは椅子の技術的詳細と材料により関心を持っています。この場合、プロンプトを継続的に修正して、椅子の技術的詳細をより正確に説明するようにできます。

解決方法：対象読者に関連する側面に焦点を当てるよう要求する。

In [13]:
# 优化后的 Prompt，说明面向对象，应具有什么性质且侧重于什么方面
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

Introducing our beautiful medieval-style office furniture collection, including file cabinets, desks, bookcases, and conference tables. Available in multiple shell colors and base coatings, with optional plastic or fabric/leather upholstery. Features a plastic-coated aluminum base with five wheels and pneumatic chair adjustment. Suitable for home or commercial use and made with high-quality materials, including cast aluminum with a modified nylon coating and HD36 foam. Made in Italy.


In [ ]:
# 最適化後のプロンプト、対象読者を説明し、どのような性質を持ち、どの側面に焦点を当てるべきかを示す
prompt = f"""
あなたのタスクは、技術仕様書に基づいて製品の小売ウェブサイト用説明を作成することで、マーケティングチームを支援することです。

三つのバッククォートで区切られた技術仕様書で提供された情報に基づいて、製品説明を書いてください。

この説明は家具小売業者向けなので、技術的な性質を持ち、製品の材料構造に焦点を当てる必要があります。

最大50語まで使用してください。

技術仕様： ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

プロンプトを修正することで、モデルの焦点が具体的な特徴と技術的詳細に傾いたことが分かります。

説明の最後に製品IDを表示したいと思うかもしれません。そこで、このプロンプトをさらに改善して、説明の最後に仕様書の7桁の製品IDを表示するよう要求できます。

In [15]:
# 更进一步，要求在描述末尾包含 7个字符的产品ID
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

At the end of the description, include every 7-character 
Product ID in the technical specification.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

Introducing our beautiful medieval-style office furniture collection, featuring file cabinets, desks, bookshelves, and conference tables. Available in multiple shell colors and base coatings, with optional plastic or fabric/leather decorations. The chair comes with or without armrests and has a plastic-coated aluminum base with five wheels and pneumatic height adjustment. Suitable for home or commercial use. Made in Italy.

Product IDs: SWC-100, SWC-110


In [ ]:
# さらに進んで
prompt = f"""
あなたのタスクは、技術仕様書に基づいて製品の小売ウェブサイト用説明を作成することで、マーケティングチームを支援することです。

三つのバッククォートで区切られた技術仕様書で提供された情報に基づいて、製品説明を書いてください。

この説明は家具小売業者向けなので、技術的な性質を持ち、製品の材料構造に焦点を当てる必要があります。

説明の最後に、技術仕様書の各7文字の製品IDを含めてください。

最大50語まで使用してください。

技術仕様： ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

以上は、多くの開発者が通常経験するプロンプト開発の反復プロセスの簡単な例です。私の提案は、前章で実演したように、プロンプトは明確で具体的であるべきで、必要に応じてモデルに考える時間を与えるべきです。これらの要件の基礎の上で、一般的なフローは最初にプロンプトの一版を書いて、何が起こるかを確認し、その後プロンプトを継続的に反復改善して、必要な結果に段階的に近づけることです。多くの成功したプロンプトは、この反復プロセスを通じて得られています。ChatGPTの能力についてより深い理解を得られるかもしれない、より複雑なプロンプトの例をお見せします。

## 2.3 問題三：表形式の説明を追加
継続して指示を追加し、製品寸法情報を抽出してテーブルに整理し、テーブルの列、テーブル名、形式を指定します。その後、すべての内容をウェブページで使用できるHTML形式にフォーマットします。

In [17]:
# 要求它抽取信息并组织成表格，并指定表格的列、表名和格式
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

At the end of the description, include every 7-character 
Product ID in the technical specification.

After the description, include a table that gives the 
product's dimensions. The table should have two columns.
In the first column include the name of the dimension. 
In the second column include the measurements in inches only.

Give the table the title 'Product Dimensions'.

Format everything as HTML that can be used in a website. 
Place the description in a <div> element.

Technical specifications: ```{fact_sheet_chair}```
"""

response = get_completion(prompt)
print(response)

<div>
  <p>Introducing our beautiful collection of medieval-style office furniture, including file cabinets, desks, bookcases, and conference tables. Choose from a variety of shell colors and base coatings. You can opt for plastic front and backrest decoration (SWC-100) or full decoration with 10 fabrics and 6 leathers (SWC-110). Base coating options include stainless steel, matte black, glossy white, or chrome. The chair is available with or without armrests and is suitable for both home and commercial settings. It is contract eligible.</p>
  <p>The structure features a plastic-coated aluminum base with five wheels. The chair is pneumatically adjustable for easy height adjustment.</p>
  <p>Product IDs: SWC-100, SWC-110</p>
  <table>
    <caption>Product Dimensions</caption>
    <tr>
      <td>Width</td>
      <td>20.87 inches</td>
    </tr>
    <tr>
      <td>Depth</td>
      <td>20.08 inches</td>
    </tr>
    <tr>
      <td>Height</td>
      <td>31.50 inches</td>
    </tr>
    <tr>


In [18]:
# 表格是以 HTML 格式呈现的，加载出来
from IPython.display import display, HTML

display(HTML(response))

Width,20.87 inches
Depth,20.08 inches
Height,31.50 inches
Seat Height,17.32 inches
Seat Depth,16.14 inches


In [ ]:
# 情報を抽出してテーブルに整理し、テーブルの列、テーブル名、形式を指定するよう要求
prompt = f"""
あなたのタスクは、技術仕様書に基づいて製品の小売ウェブサイト用説明を作成することで、マーケティングチームを支援することです。

三つのバッククォートで区切られた技術仕様書で提供された情報に基づいて、製品説明を書いてください。

この説明は家具小売業者向けなので、技術的な性質を持ち、製品の材料構造に焦点を当てる必要があります。

説明の最後に、技術仕様書の各7文字の製品IDを含めてください。

説明の後に、製品の寸法を示すテーブルを含めてください。テーブルは2列にする必要があります。
最初の列には寸法の名前を含めてください。
2番目の列にはインチでの測定値のみを含めてください。

テーブルのタイトルを「製品寸法」にしてください。

すべてをウェブサイトで使用できるHTML形式でフォーマットしてください。
説明を<div>要素に配置してください。

技術仕様：```{fact_sheet_chair}```
"""

response = get_completion(prompt)
print(response)

In [ ]:
# テーブルはHTML形式で表示され、ロードして表示
from IPython.display import display, HTML

display(HTML(response))

本章の主要内容は、アプリケーション開発におけるLLMの反復的プロンプト開発プロセスです。開発者はまずプロンプトを書くことを試み、その後反復を通じて段階的に改善し、必要な結果を得るまで続ける必要があります。効率的なプロンプトエンジニア（Prompt Engineer）として、重要なのは効果的なプロンプト開発プロセスを習得することであり、「完璧な」プロンプトを求めることではありません。より複雑なアプリケーションの場合、複数のサンプル（数百の仕様書など）でプロンプトの反復開発を行い、サンプルセット上で評価することができます。

最後に、より成熟したアプリケーションでは、複数のサンプルセットでの複数のプロンプトの性能を観察し、平均または最悪の性能をテストできます。ただし、通常、**アプリケーションがより成熟した後にのみ**、このような評価方法で継続的改善を図ることをお勧めします。

Jupyter Notebookを使用して、本節で示した例を実際に実践し、さまざまな変更を試して結果を確認してください。